In [1]:
from app import db
from models import Company, Internship, Department, Review
import pandas as pd
import sqlalchemy

In [23]:
models = {Company: "companies",
         Internship: "internships",
         Department: "departments",
         Review: "reviews"}

In [24]:
dfs = {}
for key, value in models.items():
    rows = []
    for i in key.query.all():
        row = {}
        for k, v in i.__dict__.items():
            if type(v) != sqlalchemy.orm.state.InstanceState:
                row[k] = v
        rows.append(row)
    dfs[f'{value}'] = rows

In [25]:
companies = pd.DataFrame(dfs['companies'])
departments = pd.DataFrame(dfs['departments'])
internships = pd.DataFrame(dfs['internships'])
reviews = pd.DataFrame(dfs['reviews'])

In [26]:
companies.rename(columns={'id': 'company_id'}, inplace=True)
departments.rename(columns={'id': 'department_id'}, inplace=True)
internships.rename(columns={'id': 'internship_id'}, inplace=True)

In [28]:
reviews = pd.merge(reviews, 
                   departments.loc[:, 
                        ['company_id', 'internship_id', 
                         'department_id', 'title']], 
                   how='left', on='department_id')

In [29]:
reviews = pd.merge(reviews, internships.loc[:,
                                ['internship_id', 'title']],
                  how='left', on='internship_id')

In [30]:
reviews = pd.merge(reviews, companies.loc[:,
                                ['company_id', 'name']],
                  how='left', on='company_id')

In [31]:
reviews.rename(columns={'title_x': 'deparment',
                       'title_y': 'internship',
                       'name': 'company'}, inplace=True)

In [39]:
t = {}
for i in reviews.loc[:, 'department_id'].values:
    if i not in t:
        t[i] = 1
    else:
        t[i] += 1 
t

{184: 1,
 185: 1,
 138: 2,
 188: 2,
 175: 1,
 132: 2,
 9: 1,
 191: 1,
 193: 1,
 194: 1,
 28: 1,
 63: 1,
 15: 1,
 160: 1,
 2: 1,
 45: 1,
 172: 1,
 133: 1,
 48: 1}

In [41]:
reviews[reviews.department_id == 138]

,cons,contract,date_posted,department_id,id,mentorshipRating,month_end,month_start,paid,pros,...,usefulnessRating,workContinued,workType,year_end,year_start,company_id,internship_id,deparment,internship,company
2,Отношение к стажерам как к расходному материалу,1,2019-05-23 19:34:44.024781,138,10,3,март,октябрь,1,Легко отпроситься на контрольную/защиту курсов...,...,4,0,1,2018,2017,24,29,Аудит,Стажировка,PwC
17,,1,2019-06-10 11:00:39.743091,138,25,5,июнь,февраль,0,,...,5,0,1,2018,2018,24,29,Аудит,Стажировка,PwC


In [36]:
internships.shape

(44, 3)

In [37]:
departments.shape

(195, 9)